In [1]:
!pip install transformers peft bitsandbytes trl deepeval optimum auto-gptq langchain torch flask
!pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.4/418.4 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 37.6 MB/s eta 0:00:00
     ━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig
from trl import SFTTrainer

#Quantize 4bits

In [4]:
compute_dtypes = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtypes,
)

In [1]:
# from google.colab import userdata
# userdata.get('Llama-3-8B')

In [6]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [7]:
# Ensure local directory does not exist
!rm -rf meta-llama/Meta-Llama-3-8B

model_id = "meta-llama/Meta-Llama-3-8B"
access_token = "#"

# Pass the access token to the tokenizer as well
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_id, token = access_token, quantization_config = quant_config,  device_map={"": 0})

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=500)
hf = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [8]:
prompt_template = PromptTemplate(
    input_variables=["product_name", "product_description", "interest_level", "user_name", "tone"],
    template="""
Write a personalized marketing email in a paragraph form with the following details, but do not generate information about sales, promotions, discounts, prices, discount codes, or dates:

Product Name: {product_name}
Product Description: {product_description}
Interest Level: {interest_level}
User Name: {user_name}
Email Tone: {tone}

Email Content in Paragraph:
Subject: Exclusive {product_name} Offer Just for You!

Dear {user_name},
"""
)

email_chain = LLMChain(llm=hf, prompt=prompt_template)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
import re
# Define the function to generate the email
def generate_email_auto(product_name, product_description, interest_level, user_name):
    # Fixed template values
    sender_name = "Binit Sapkota"
    company_name = "MrBeerGod Technologies"
    website_url = "https://barahsinghe.com/"
    validity_date = "1 DECEMBER 2024"

    # Determine discount offer and coupon code based on interest level
    discount_map = {
        "very interested": ("30%", "NEPAL30"),
        "interested": ("20%", "NEPAL20"),
        "somewhat interested": ("15%", "NEPAL15"),
        "not interested": ("10%", "NEPAL10")
    }
    discount_offers, coupon_code = discount_map.get(interest_level, ("10%", "NEPAL10"))  # Default

    # Determine email tone based on interest level
    tone_map = {
        "very interested": "excited and convincing",
        "interested": "enthusiastic",
        "somewhat interested": "informative",
        "not interested": "gentle reminder"
    }
    tone = tone_map.get(interest_level, "informative")

    # Structured input for the email generation
    structured_input = (
        f"Generate a personalized email in paragraph form with the following details, but do not generate information about sales, promotions, discounts, prices of the product, discount codes, or dates:\n"
        f"Product Name: {product_name}\n"
        f"Product Description: {product_description}\n"
        f"Interest Level: {interest_level}\n"
        f"User Name: {user_name}\n"
        f"Email Tone: {tone}\n"
        f"Email Content in Paragraph:\nSubject: Exclusive {product_name} Offer Just for You!\n\n\n"
        f"Dear {user_name},\n\n\n"
    )

    # Tokenize the structured input
    inputs = tokenizer(structured_input, return_tensors="pt")

    while True:  # Loop for validation
        # Generate the output
        outputs = model.generate(
            inputs.input_ids.to('cuda'),
            max_length=500,
            min_length=200,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=3,
        )

        # Decode and process the email
        email = tokenizer.decode(outputs[0], skip_special_tokens=True)
        email_content_start = email.find("Subject:")
        generated_email = email[email_content_start:].strip()


        # Remove any existing sign-offs
        sign_offs = ["Regards,", "Best regards,", "Sincerely,", "Yours truly,", "Best", "money would be refunded", "company would be liable", "product can be returned after used", "simply use the discount"]
        for sign_off in sign_offs:
            if sign_off in generated_email:
                generated_email = generated_email.split(sign_off)[0].strip()

        # Guard against generated coupon codes
        forbidden_phrases = ["code", "coupon code", "discount code", "offer code", "Discount Offer", "retailed price", "retailed rate","discounted price", "discounted rate", "price", "product price", " 30-day money-back guarantee"]
        for phrase in forbidden_phrases:
            if phrase in generated_email:
                generated_email = generated_email.split(phrase)[0].strip()


        # Filter and remove unwanted elements
        generated_email = re.sub(r'http\S+', '', generated_email)
        generated_email = re.sub(r'www\.\S+', '', generated_email)
        generated_email = re.sub(r'<.*?>', '', generated_email)
        generated_email = re.sub(r'\d+% off', '', generated_email)

        # Append template values with dynamic coupon code and discount offer
        generated_email += (
            f"\n\nUse the coupon code '{coupon_code}' to get {discount_offers} off. Valid till {validity_date}.\n"
            f"\nBest regards,\n"
            f"{sender_name}\n"
            f"{company_name}\n"
            f"Visit us: {website_url}\n"
        )

        # Validation check
        if is_valid_email(generated_email):
            return generated_email
        else:
            print("Email not valid. Regenerating...")



In [12]:
def is_valid_email(email):
    # Customization based on my requirement
    forbidden_phrases = ["money would be refunded", "company would be liable",
                         "product can be returned after used", "simply use the discount", "sales", "promotions", "discounts", "prices", "price" ,  "discount code", "discount" , "dates", "Expiration Date" , "regular price", "Sincerely", "The SuperWidgets Team", "Yours sincerely", "Name of Company"]
    if any(phrase in email for phrase in forbidden_phrases):
        return False
    if len(email.split()) < 100 or len(email.split()) > 200:
        return False
    return True



In [13]:
# example for the generation
product_name = "SuperWidget"
product_description = "The SuperWidget is an innovative tool that helps you increase productivity and efficiency."
interest_level = "very interested"
user_name = "John Doe"

personalized_email = generate_email_auto(product_name, product_description, interest_level, user_name)
print(personalized_email)

Email not valid. Regenerating...
Email not valid. Regenerating...
Email not valid. Regenerating...
Email not valid. Regenerating...
Subject: Exclusive SuperWidget Offer Just for You!


Dear John Doe,


We are excited to offer you our exclusive SuperWidget offer! Our SuperWidget has been designed to help you increase your productivity and save time. It is the perfect tool for busy professionals like you who are always on the go.

Our SuperWidget comes with a variety of features, including:

1. A sleek and modern design that fits perfectly in your hand.
2. An easy-to-use interface that allows you to quickly access the features you need.
3. A powerful battery that lasts up to 10 hours on a single charge.
4. A wide range of accessories, such as a carrying case and charging cable, to keep your SuperWidget protected and ready to go.

We are confident that our SuperWidget will help you improve your productivity, efficiency, and overall quality of life. Don't miss out on this exclusive offer -

In [14]:

# Define the function to generate the email
def generate_email_auto(product_name, product_description, interest_level, user_name):
    # Fixed template values
    sender_name = "Binit Sapkota"
    company_name = "MrBeerGod Technologies"
    website_url = "https://barahsinghe.com/"
    validity_date = "1 DECEMBER 2024"

    # Determine discount offer and coupon code based on interest level
    discount_map = {
        "very interested": ("30%", "NEPAL30"),
        "interested": ("20%", "NEPAL20"),
        "somewhat interested": ("15%", "NEPAL15"),
        "not interested": ("10%", "NEPAL10")
    }
    discount_offers, coupon_code = discount_map.get(interest_level, ("10%", "NEPAL10"))  # Default

    # Determine email tone based on interest level
    tone_map = {
        "very interested": "excited and convincing",
        "interested": "enthusiastic",
        "somewhat interested": "informative",
        "not interested": "gentle reminder"
    }
    tone = tone_map.get(interest_level, "informative")

    # Generate the email content
    email_content = email_chain.run(
        product_name=product_name,
        product_description=product_description,
        interest_level=interest_level,
        user_name=user_name,
        tone=tone
    )

    # Post-process the generated email content
    email_content = email_content.strip()

    # Append additional personalized closing
    personalized_email = f"""
{email_content}

Use the coupon code '{coupon_code}' to get {discount_offers} off. Valid till {validity_date}.

Best regards,
{sender_name}
{company_name}
Visit us: {website_url}
"""
    return personalized_email

# Example usage for the email generation
product_name = "SuperWidget"
product_description = "The SuperWidget is an innovative tool that helps you increase productivity and efficiency."
interest_level = "very interested"
user_name = "John Doe"

personalized_email = generate_email_auto(product_name, product_description, interest_level, user_name)
print(personalized_email)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Write a personalized marketing email in a paragraph form with the following details, but do not generate information about sales, promotions, discounts, prices, discount codes, or dates:

Product Name: SuperWidget
Product Description: The SuperWidget is an innovative tool that helps you increase productivity and efficiency.
Interest Level: very interested
User Name: John Doe
Email Tone: excited and convincing

Email Content in Paragraph:
Subject: Exclusive SuperWidget Offer Just for You!

Dear John Doe,
We are excited to offer you an exclusive offer on our SuperWidget. As a valued customer, we want to provide you with the best possible experience and ensure that you are getting the most out of our product.
The SuperWidget is an innovative tool that helps you increase productivity and efficiency. It is designed to help you stay organized, manage your tasks, and keep track of your progress. With its intuitive interface and user-friendly design, the SuperWidget is a must-have for any pro